## 0. Libarary 불러오기 및 경로설정

In [1]:
import torch
from sklearn.metrics import confusion_matrix, recall_score, f1_score
from tqdm import tqdm
from importlib import import_module
import multiprocessing
from dataset import BaseAugmentation

In [2]:
data_dir = '/data/ephemeral/home/data/train/images'

In [9]:
# dataset 임포트
from dataset import AgeModelDataset

# dataset 불러오기
dataset = AgeModelDataset(data_dir)
class MyArg:
    resize = [128, 96]

# transform 임포트
transform = BaseAugmentation(MyArg(), dataset)
dataset.set_transform(transform)

#
age_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=64,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=True,
        pin_memory=torch.cuda.is_available(),
        drop_last=False,
    )

/opt/conda/envs/level1/lib/python3.10/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [10]:
# model import
from models.age_model import SingleResNet50

# model 불러오기
model = SingleResNet50(3)

model_loc = './BEST_age_model_expCutMix300epoch/age_model_best.pth'

model.load_state_dict(torch.load(model_loc, map_location='cuda'))
model.eval()

labels = []
preds = []
for index, batch in enumerate(tqdm(age_loader)):
    image, label = batch
    image = image
    label = label.long()
    labels.extend(label.numpy())
    pred = model(image) 
    preds.extend(torch.argmax(pred, dim=-1))
    # 아래 주석을 사용하면 빠른 결과, 부정확한 score. (약 1분 소요)
    # 그렇지 않으면 느린 결과, 정확한 score. (약 10분 소요)
    # if index > 20:
    #     break

/opt/conda/envs/level1/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/level1/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 1/1012 [00:06<1:43:36,  6.15s/it]

In [29]:
print(preds)
print(labels)

[tensor(1), tensor(0), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(1), tensor(2), tensor(0), tensor(0), tensor(2), tensor(0), tensor(2), tensor(0), tensor(2), tensor(1), tensor(2), tensor(2), tensor(0), tensor(0), tensor(0), tensor(1), tensor(1), tensor(2), tensor(0), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(0), tensor(0), tensor(1), tensor(1), tensor(0), tensor(0), tensor(0), tensor(1), tensor(2), tensor(0), tensor(1), tensor(2), tensor(1), tensor(2), tensor(1), tensor(0), tensor(2), tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(2), tensor(1), tensor(1), tensor(0), tensor(1), tensor(0), tensor(0), tensor(1), tensor(1), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(1), tensor(1), tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(2), tensor(1), tensor(0), tensor(2), tensor(2), tensor(1), tensor(0)

In [30]:
recall = recall_score(labels, preds, average='macro')
f1 = f1_score(labels, preds, average='macro')
conf_matrix = confusion_matrix(labels, preds)

print("Recall:", recall)
print("f1:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Recall: 0.9588768115942029
f1: 0.9682099466053434
Confusion Matrix:
[[182   2   0]
 [  0 176   0]
 [  0   9  71]]


In [27]:
torch.save(model.state_dict(), './KANGSH_age_model_best.pth')